# Quick Start - Run local code as SageMaker training job


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

---


We're introducing a new capability in AWS SageMaker Python SDK that allows data scientists to run their Python functions as SageMaker Jobs to take advantage of the compute power offered by SageMaker. This sample notebook is a quick introduction to this new capability with dummy Python functions.

For more details about the feature, please checkout the AWS Developer Guide [here](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator.html)


## Install the dependencies

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install -r ./requirements.txt

In [3]:
!pwd

/root


In [4]:
!ls -la

total 232
drwx------ 9 root nogroup   6144 Jun 11 09:47 .
drwxr-xr-x 1 root root        50 Jun 11 09:47 ..
drwxr-xr-x 3 root root      6144 Jun 11 05:48 .aws
drwxr-xr-x 4 root root      6144 Jun 11 05:57 .cache
drwx------ 3 root root      6144 Jun 11 05:55 .config
drwxr-xr-x 2 root root      6144 Jun 11 05:57 .ipynb_checkpoints
drwxr-xr-x 5 root root      6144 Jun 11 05:57 .ipython
drwxr-xr-x 3 root root      6144 Jun 11 05:56 .jupyter
drwxr-xr-x 3 root root      6144 Jun 11 05:48 .local
-rw-r--r-- 1 root root         2 Jun 11 09:24 .sagemaker-jumpstart-tasks-status.json
-rw-r--r-- 1 root root       215 Jun 11 06:18 .wget-hsts
-rw-r--r-- 1 root root       211 Jun 11 09:24 .yarnrc
-rw-r--r-- 1 root root    155082 Jun 11 06:18 config.yaml
-rw-r--r-- 1 root root       126 Jun 11 06:16 environment.yml
-rw-r--r-- 1 root root     27274 Jun 11 09:47 quick_start.ipynb
-rw-r--r-- 1 root root        27 Jun 11 09:43 requirements.txt


## Setup Configuration file path
We need set the directory in which the config.yaml file resides so that remote decorator can make use of the settings.

In [5]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

In [6]:
os.getcwd()

'/root'

In [7]:
import sagemaker

sagemaker.__version__

'2.156.0'

## Invoke Python function as a SageMaker job

There are two ways users could invoke a function as a job.

* Using the decorator. When the decorated function is invoked, it executes synchronously

In [9]:
from sagemaker.remote_function import remote


@remote
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


divide(3, 2)

[Sagemaker Config - applied value]
 config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
 config value that will be used = ./requirements.txt

[Sagemaker Config - applied value]
 config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType
 config value that will be used = ml.m5.xlarge



2023-06-11 09:51:59,575 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpcscxpwph/temp_workspace/sagemaker_remote_function_workspace/requirements.txt'
2023-06-11 09:51:59,577 sagemaker.remote_function INFO     Successfully created workdir archive at '/tmp/tmpcscxpwph/workspace.zip'
2023-06-11 09:51:59,624 sagemaker.remote_function INFO     Successfully uploaded workdir to 's3://sagemaker-us-east-1-075912829265/divide-2023-06-11-09-51-59-314/sm_rf_user_ws/workspace.zip'
2023-06-11 09:51:59,626 sagemaker.remote_function INFO     Serializing function code to s3://sagemaker-us-east-1-075912829265/divide-2023-06-11-09-51-59-314/function
2023-06-11 09:51:59,715 sagemaker.remote_function INFO     Serializing function arguments to s3://sagemaker-us-east-1-075912829265/divide-2023-06-11-09-51-59-314/arguments
2023-06-11 09:51:59,774 sagemaker.remote_function INFO     Creating job: divide-2023-06-11-09-51-59-314


2023-06-11 09:51:59 Starting - Starting the training job...
2023-06-11 09:52:16 Starting - Preparing the instances for training......
2023-06-11 09:53:12 Downloading - Downloading input data...
2023-06-11 09:53:43 Training - Training image download completed. Training in progress.INFO: CONDA_PKGS_DIRS is set to '/opt/ml/sagemaker/warmpoolcache/sm_remotefunction_user_dependencies_cache/conda/pkgs'
INFO: PIP_CACHE_DIR is set to '/opt/ml/sagemaker/warmpoolcache/sm_remotefunction_user_dependencies_cache/pip'
INFO: Bootstraping runtime environment.
2023-06-11 09:53:49,972 sagemaker.remote_function INFO     Successfully unpacked workspace archive at '/'.
2023-06-11 09:53:49,972 sagemaker.remote_function INFO     '/sagemaker_remote_function_workspace/pre_exec.sh' does not exist. Assuming no pre-execution commands to run
2023-06-11 09:53:49,972 sagemaker.remote_function INFO     Running command: '/usr/local/bin/python -m pip install -r /sagemaker_remote_function_workspace/requirements.txt' in 

1.5

* Using the RemoteExecutor APIs. They follow the pattern of the Python [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) APIs.

In [ ]:
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


from sagemaker.remote_function import RemoteExecutor

with RemoteExecutor(max_parallel_jobs=1, keep_alive_period_in_seconds=30) as executor:
    futures = [executor.submit(divide, x, 2) for x in [1, 2, 3]]

In [ ]:
[future.result() for future in futures]

## Customize the decorator settings

In the examples above, the @remote decorator and RemoteExecutor looked for configurations in the path set in the environment variable `SAGEMAKER_USER_CONFIG_OVERRIDE`/
We set the value to use the local configuration file `config.yaml` present in the current directory `./`

You can override the configurations by specifying the decorator arguments directly. In the following example, instead of launching the job with `ml.m5.xlarge`,
as specified in the `./config.yaml`, run the function with a more powerful instance

In [ ]:
@remote(instance_type="ml.m5.4xlarge")
def divide(x, y):
    print(f"Calculating {x}/{y}")
    return x / y


divide(3, 2)

## Add extra dependencies using conda environment yml file

In the example below, the function will run in a new conda environment where pandas and sagemaker will be installed.
(Note that the cell will not run if executed in SageMaker Studio.)

In [ ]:
import pandas as pd


@remote(dependencies="./environment.yml")
def multiply(dataframe: pd.DataFrame, factor: float):
    return dataframe * factor


df = pd.DataFrame(
    {
        "A": [14, 4, 5, 4, 1],
        "B": [5, 2, 54, 3, 2],
        "C": [20, 20, 7, 3, 8],
        "D": [14, 3, 6, 2, 6],
    }
)

multiply(df, 10.0)

## Common errors

### SerializationError and DeserializationError

Behind the scenes, the function and function arguments and returns are serialized and deserialized using Pickle.

While SerializationError occurs when passing an unpickle-able object as function argument, such as XGBoost DMatrix, open file object. DeserializationError typically happens when there is discrepancy between the dependencies in the local environments and dependencies in
the job environments. In the following example, the latest pandas is used to declare a dataframe. The dataframe is passed to the function
call. On the job side, an older version of pandas is installed. The dataframe can't be deserialized.

In [ ]:
import pandas as pd


@remote(dependencies="./incompatible_requirements.txt")
def multiply(dataframe: pd.DataFrame, factor: float):
    return dataframe * factor


df = pd.DataFrame(
    {
        "A": [14, 4, 5, 4, 1],
        "B": [5, 2, 54, 3, 2],
        "C": [20, 20, 7, 3, 8],
        "D": [14, 3, 6, 2, 6],
    }
)

multiply(df, 10.0)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker-remote-function|quick_start|quick_start.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker-remote-function|quick_start|quick_start.ipynb)
